In [3]:
from rich import print
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [4]:
# this is for understanding how the chunking is happening 
# obtained from the first chapter of SLP 
text = """
## About Regular Expressions
One of the unsung successes in standardization in computer science has been the regular expression (often shortened to **regex**), a language for specifying text search regular expression strings. This practical language is used in every computer language, word processor, and text processing tools like the Unix tools grep or Emacs. Formally, a regular expression is an algebraic notation for characterizing a set of strings. Regular expressions are particularly useful for searching in texts, when we have a **pattern** to search for and a **corpus** of texts to search through. A regular expression search function corpus will search through the corpus, returning all texts that match the pattern. The corpus can be a single document or a collection. For example, the Unix command-line tool grep takes a regular expression and returns every line of the input document that matches the expression.

A search can be designed to return every match on a line, if there are more than one, or just the first match. In the following examples we generally underline the exact part of the pattern that matches the regular expression and show only the first match. We'll show regular expressions delimited by slashes but note that slashes are not part of the regular expressions.

Regular expressions come in many variants. We'll be describing extended regular expressions; different regular expression parsers may only recognize subsets of these, or treat some expressions slightly differently. Using an online regular expression tester is a handy way to test out your expressions and explore these variations.
"""

In [6]:
# 1. Character Text Splitting
print("#### Character Text Splitting ####")

# Manual Splitting
chunks = []
chunk_size = 35 # Characters

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)

documents = [Document(page_content=chunk, metadata={"source": "slp"}) for chunk in chunks]
print(documents)

#### Character Text Splitting ####

[
    Document(page_content='\n## About Regular Expressions\nOne o', metadata={'source': 'slp'}),
    Document(page_content='f the unsung successes in standardi', metadata={'source': 'slp'}),
    Document(page_content='zation in computer science has been', metadata={'source': 'slp'}),
    Document(page_content=' the regular expression (often shor', metadata={'source': 'slp'}),
    Document(page_content='tened to **regex**), a language for', metadata={'source': 'slp'}),
    Document(page_content=' specifying text search regular exp', metadata={'source': 'slp'}),
    Document(page_content='ression strings. This practical lan', metadata={'source': 'slp'}),
    Document(page_content='guage is used in every computer lan', metadata={'source': 'slp'}),
    Document(page_content='guage, word processor, and text pro', metadata={'source': 'slp'}),
    Document(page_content='cessing tools like the Unix tools g', metadata={'source': 'slp'}),
    Document(page_content='rep or Emacs. Formally, a regular e', metadata={'source': 'slp'}),
    Document(page_content='xpression is an algebraic notation ', metadata={'source': 'slp'}),
    Document(page_content='for characterizing a set of strings', metadata={'source': 'slp'}),
    Document(page_content='. Regular expressions are particula', metadata={'source': 'slp'}),
    Document(page_content='rly useful for searching in texts, ', metadata={'source': 'slp'}),
    Document(page_content='when we have a **pattern** to searc', metadata={'source': 'slp'}),
    Document(page_content='h for and a **corpus** of texts to ', metadata={'source': 'slp'}),
    Document(page_content='search through. A regular expressio', metadata={'source': 'slp'}),
    Document(page_content='n search function corpus will searc', metadata={'source': 'slp'}),
    Document(page_content='h through the corpus, returning all', metadata={'source': 'slp'}),
    Document(page_content=' texts that match the pattern. The ', metadata={'source': 'slp'}),
    Document(page_content='corpus can be a single document or ', metadata={'source': 'slp'}),
    Document(page_content='a collection. For example, the Unix', metadata={'source': 'slp'}),
    Document(page_content=' command-line tool grep takes a reg', metadata={'source': 'slp'}),
    Document(page_content='ular expression and returns every l', metadata={'source': 'slp'}),
    Document(page_content='ine of the input document that matc', metadata={'source': 'slp'}),
    Document(page_content='hes the expression.\n\nA search can b', metadata={'source': 'slp'}),
    Document(page_content='e designed to return every match on', metadata={'source': 'slp'}),
    Document(page_content=' a line, if there are more than one', metadata={'source': 'slp'}),
    Document(page_content=', or just the first match. In the f', metadata={'source': 'slp'}),
    Document(page_content='ollowing examples we generally unde', metadata={'source': 'slp'}),
    Document(page_content='rline the exact part of the pattern', metadata={'source': 'slp'}),
    Document(page_content=' that matches the regular expressio', metadata={'source': 'slp'}),
    Document(page_content='n and show only the first match. We', metadata={'source': 'slp'}),
    Document(page_content="'ll show regular expressions delimi", metadata={'source': 'slp'}),
    Document(page_content='ted by slashes but note that slashe', metadata={'source': 'slp'}),
    Document(page_content='s are not part of the regular expre', metadata={'source': 'slp'}),
    Document(page_content='ssions.\n\nRegular expressions come i', metadata={'source': 'slp'}),
    Document(page_content="n many variants. We'll be describin", metadata={'source': 'slp'}),
    Document(page_content='g extended regular expressions; dif', metadata={'source': 'slp'}),
    Document(page_content='ferent regular expression parsers m', metadata={'source': 'slp'}),
    Document(page_content='ay only recognize subsets of these,', metadata={'source': 'slp'}),
    Document(page_content=' or treat some ex

In [7]:
# Automatic Text Splitting
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=35, 
    chunk_overlap=0, 
    separator='', 
    strip_whitespace=False
)

documents = text_splitter.create_documents([text])
print(documents)

[
    Document(page_content='\n## About Regular Expressions\nOne o'),
    Document(page_content='f the unsung successes in standardi'),
    Document(page_content='zation in computer science has been'),
    Document(page_content=' the regular expression (often shor'),
    Document(page_content='tened to **regex**), a language for'),
    Document(page_content=' specifying text search regular exp'),
    Document(page_content='ression strings. This practical lan'),
    Document(page_content='guage is used in every computer lan'),
    Document(page_content='guage, word processor, and text pro'),
    Document(page_content='cessing tools like the Unix tools g'),
    Document(page_content='rep or Emacs. Formally, a regular e'),
    Document(page_content='xpression is an algebraic notation '),
    Document(page_content='for characterizing a set of strings'),
    Document(page_content='. Regular expressions are particula'),
    Document(page_content='rly useful for searching in texts, '),
    Document(page_content='when we have a **pattern** to searc'),
    Document(page_content='h for and a **corpus** of texts to '),
    Document(page_content='search through. A regular expressio'),
    Document(page_content='n search function corpus will searc'),
    Document(page_content='h through the corpus, returning all'),
    Document(page_content=' texts that match the pattern. The '),
    Document(page_content='corpus can be a single document or '),
    Document(page_content='a collection. For example, the Unix'),
    Document(page_content=' command-line tool grep takes a reg'),
    Document(page_content='ular expression and returns every l'),
    Document(page_content='ine of the input document that matc'),
    Document(page_content='hes the expression.\n\nA search can b'),
    Document(page_content='e designed to return every match on'),
    Document(page_content=' a line, if there are more than one'),
    Document(page_content=', or just the first match. In the f'),
    Document(page_content='ollowing examples we generally unde'),
    Document(page_content='rline the exact part of the pattern'),
    Document(page_content=' that matches the regular expressio'),
    Document(page_content='n and show only the first match. We'),
    Document(page_content="'ll show regular expressions delimi"),
    Document(page_content='ted by slashes but note that slashe'),
    Document(page_content='s are not part of the regular expre'),
    Document(page_content='ssions.\n\nRegular expressions come i'),
    Document(page_content="n many variants. We'll be describin"),
    Document(page_content='g extended regular expressions; dif'),
    Document(page_content='ferent regular expression parsers m'),
    Document(page_content='ay only recognize subsets of these,'),
    Document(page_content=' or treat some expressions slightly'),
    Document(page_content=' differently. Using an online regul'),
    Document(page_content='ar expression tester is a handy way'),
    Document(page_content=' to test out your expressions and e'),
    Document(page_content='xplore these variations.\n')
]

In [8]:
# 2. Recursive Character Text Splitting
print("#### Recursive Character Text Splitting ####")

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 65, 
    chunk_overlap=0
) # ["\n\n", "\n", " ", ""] 65,450

print(text_splitter.create_documents([text]))

#### Recursive Character Text Splitting ####

[
    Document(page_content='## About Regular Expressions'),
    Document(page_content='One of the unsung successes in standardization in computer'),
    Document(page_content='science has been the regular expression (often shortened to'),
    Document(page_content='**regex**), a language for specifying text search regular'),
    Document(page_content='expression strings. This practical language is used in every'),
    Document(page_content='computer language, word processor, and text processing tools'),
    Document(page_content='like the Unix tools grep or Emacs. Formally, a regular'),
    Document(page_content='expression is an algebraic notation for characterizing a set of'),
    Document(page_content='strings. Regular expressions are particularly useful for'),
    Document(page_content='searching in texts, when we have a **pattern** to search for and'),
    Document(page_content='a **corpus** of texts to search through. A regular expression'),
    Document(page_content='search function corpus will search through the corpus, returning'),
    Document(page_content='all texts that match the pattern. The corpus can be a single'),
    Document(page_content='document or a collection. For example, the Unix command-line'),
    Document(page_content='tool grep takes a regular expression and returns every line of'),
    Document(page_content='the input document that matches the expression.'),
    Document(page_content='A search can be designed to return every match on a line, if'),
    Document(page_content='there are more than one, or just the first match. In the'),
    Document(page_content='following examples we generally underline the exact part of the'),
    Document(page_content='pattern that matches the regular expression and show only the'),
    Document(page_content="first match. We'll show regular expressions delimited by slashes"),
    Document(page_content='but note that slashes are not part of the regular expressions.'),
    Document(page_content="Regular expressions come in many variants. We'll be describing"),
    Document(page_content='extended regular expressions; different regular expression'),
    Document(page_content='parsers may only recognize subsets of these, or treat some'),
    Document(page_content='expressions slightly differently. Using an online regular'),
    Document(page_content='expression tester is a handy way to test out your expressions'),
    Document(page_content='and explore these variations.')
]

In [9]:

# 3. Document Specific Splitting
print("#### Document Specific Splitting ####")

# Document Specific Splitting - Markdown
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

documents = splitter.split_text(text)
print(documents)

#### Document Specific Splitting ####

[
    Document(
        page_content="One of the unsung successes in standardization in computer science has been the regular 
expression (often shortened to **regex**), a language for specifying text search regular expression strings. This 
practical language is used in every computer language, word processor, and text processing tools like the Unix 
tools grep or Emacs. Formally, a regular expression is an algebraic notation for characterizing a set of strings. 
Regular expressions are particularly useful for searching in texts, when we have a **pattern** to search for and a 
**corpus** of texts to search through. A regular expression search function corpus will search through the corpus, 
returning all texts that match the pattern. The corpus can be a single document or a collection. For example, the 
Unix command-line tool grep takes a regular expression and returns every line of the input document that matches 
the expression.  \nA search can be designed to return every match on a line, if there are more than one, or just 
the first match. In the following examples we generally underline the exact part of the pattern that matches the 
regular expression and show only the first match. We'll show regular expressions delimited by slashes but note that
slashes are not part of the regular expressions.  \nRegular expressions come in many variants. We'll be describing 
extended regular expressions; different regular expression parsers may only recognize subsets of these, or treat 
some expressions slightly differently. Using an online regular expression tester is a handy way to test out your 
expressions and explore these variations.",
        metadata={'Header 2': 'About Regular Expressions'}
    )
]

In [10]:
# 4. Semantic Chunking
print("#### Semantic Chunking ####")

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Percentile - all differences between sentences are calculated, and then any difference greater than the X percentile is split
text_splitter = SemanticChunker(OpenAIEmbeddings())
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), 
    breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
documents = text_splitter.create_documents([text])
print(documents)

#### Semantic Chunking ####

[
    Document(
        page_content="\n## About Regular Expressions\nOne of the unsung successes in standardization in computer 
science has been the regular expression (often shortened to **regex**), a language for specifying text search 
regular expression strings. This practical language is used in every computer language, word processor, and text 
processing tools like the Unix tools grep or Emacs. Formally, a regular expression is an algebraic notation for 
characterizing a set of strings. Regular expressions are particularly useful for searching in texts, when we have a
**pattern** to search for and a **corpus** of texts to search through. A regular expression search function corpus 
will search through the corpus, returning all texts that match the pattern. The corpus can be a single document or 
a collection. For example, the Unix command-line tool grep takes a regular expression and returns every line of the
input document that matches the expression. A search can be designed to return every match on a line, if there are 
more than one, or just the first match. In the following examples we generally underline the exact part of the 
pattern that matches the regular expression and show only the first match. We'll show regular expressions delimited
by slashes but note that slashes are not part of the regular expressions. Regular expressions come in many 
variants."
    ),
    Document(
        page_content="We'll be describing extended regular expressions; different regular expression parsers may 
only recognize subsets of these, or treat some expressions slightly differently. Using an online regular expression
tester is a handy way to test out your expressions and explore these variations. "
    )
]

In [11]:
# 5. Agentic Chunking
print("#### Proposition-Based Chunking ####")

# Paper: https://arxiv.org/pdf/2312.06648.pdf

from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub

obj = hub.pull("wfh/proposal-indexing")

# define which llm to use for propositional chunking 
MODEL = "gpt-4-0125-preview"
llm = ChatOpenAI(model=MODEL)

runnable = obj | llm

class Sentences(BaseModel):
    sentences: List[str]

# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content
    propositions = extraction_chain.invoke(runnable_output)["text"][0].sentences
    return propositions

text = text.strip()
paragraphs = text.split("\n\n")

print(paragraphs)

text_propositions = []
for i, para in enumerate(paragraphs):
    propositions = get_propositions(para)
    text_propositions.extend(propositions)
    print (f"Done with {i}")

print(text_propositions)
print (f"You have {len(text_propositions)} propositions")

print("#### Agentic Chunking ####")

from agenticchunker import AgenticChunker

ac = AgenticChunker()
ac.add_propositions(text_propositions)
print(ac.pretty_print_chunks())
chunks = ac.get_chunks(get_type='list_of_strings')
print(chunks)

documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]

print(documents)

#### Proposition-Based Chunking ####

[
    '## About Regular Expressions\nOne of the unsung successes in standardization in computer science has been the 
regular expression (often shortened to **regex**), a language for specifying text search regular expression 
strings. This practical language is used in every computer language, word processor, and text processing tools like
the Unix tools grep or Emacs. Formally, a regular expression is an algebraic notation for characterizing a set of 
strings. Regular expressions are particularly useful for searching in texts, when we have a **pattern** to search 
for and a **corpus** of texts to search through. A regular expression search function corpus will search through 
the corpus, returning all texts that match the pattern. The corpus can be a single document or a collection. For 
example, the Unix command-line tool grep takes a regular expression and returns every line of the input document 
that matches the expression.',
    "A search can be designed to return every match on a line, if there are more than one, or just the first match.
In the following examples we generally underline the exact part of the pattern that matches the regular expression 
and show only the first match. We'll show regular expressions delimited by slashes but note that slashes are not 
part of the regular expressions.",
    "Regular expressions come in many variants. We'll be describing extended regular expressions; different regular
expression parsers may only recognize subsets of these, or treat some expressions slightly differently. Using an 
online regular expression tester is a handy way to test out your expressions and explore these variations."
]

Done with 0

Done with 1

Done with 2

...

-The corpus can be a single document.

-The corpus can be a collection.

None

[
    'One of the unsung successes in standardization in computer science has been the regular expression.',
    'Regular expression is often shortened to regex. Regular expression is a language for specifying text search 
regular expression strings. Regular expression is used in every computer language. Regular expression is used in 
every word processor. Regular expression is used in text processing tools like the Unix tools grep or Emacs. 
Formally, a regular expression is an algebraic notation for characterizing a set of strings. Regular expressions 
are particularly useful for searching in texts. Regular expressions are used when there is a pattern to search for.
Regular expressions are used when there is a corpus of texts to search through. A regular expression search 
function will search through the corpus. A regular expression search function returns all texts that match the 
pattern. The Unix command-line tool grep takes a regular expression. The Unix command-line tool grep returns every 
line of the input document that matches the expression. A search can be designed to return every match on a line if
there are more than one. A search can be designed to return just the first match on a line. In the following 
examples, the exact part of the pattern that matches the regular expression is generally underlined. In the 
following examples, only the first match is shown. Regular expressions in the examples are delimited by slashes. 
Slashes are not part of the regular expressions. Regular expressions come in many variants. We will be describing 
extended regular expressions. Different regular expression parsers may only recognize subsets of extended regular 
expressions. Different regular expression parsers may treat some expressions slightly differently. Using an online 
regular expression tester is a handy way to test out your regular expressions. Using an online regular expression 
tester is a handy way to explore variations of regular expressions.',
    'The corpus can be a single document. The corpus can be a collection.'
]

[
    Document(
        page_content='One of the unsung successes in standardization in computer science has been the regular 
expression.',
        metadata={'source': 'local'}
    ),
    Document(
        page_content='Regular expression is often shortened to regex. Regular expression is a language for 
specifying text search regular expression strings. Regular expression is used in every computer language. Regular 
expression is used in every word processor. Regular expression is used in text processing tools like the Unix tools
grep or Emacs. Formally, a regular expression is an algebraic notation for characterizing a set of strings. Regular
expressions are particularly useful for searching in texts. Regular expressions are used when there is a pattern to
search for. Regular expressions are used when there is a corpus of texts to search through. A regular expression 
search function will search through the corpus. A regular expression search function returns all texts that match 
the pattern. The Unix command-line tool grep takes a regular expression. The Unix command-line tool grep returns 
every line of the input document that matches the expression. A search can be designed to return every match on a 
line if there are more than one. A search can be designed to return just the first match on a line. In the 
following examples, the exact part of the pattern that matches the regular expression is generally underlined. In 
the following examples, only the first match is shown. Regular expressions in the examples are delimited by 
slashes. Slashes are not part of the regular expressions. Regular expressions come in many variants. We will be 
describing extended regular expressions. Different regular expression parsers may only recognize subsets of 
extended regular expressions. Different regular expression parsers may treat some expressions slightly differently.
Using an online regular expression tester is a handy way to test out your regular expressions. Using an online 
regular expression tester is a handy way to explore variations of regular expressions.',
        metadata={'source': 'local'}
    ),
    Document(
        page_content='The corpus can be a single document. The corpus can be a collection.',
        metadata={'source': 'local'}
    )
]

In [12]:
MODEL = "mistral"
TOP_P = 0.9
TOP_K = 100
MAX_TOKENS = 4096

local_llm = ChatOllama(
    model=MODEL,
    num_ctx=MAX_TOKENS,
    top_p=TOP_P,
    top_k=TOP_K
)

# RAG
def rag(chunks, collection_name):
    vectorstore = Chroma.from_documents(
        documents=documents,
        collection_name=collection_name,
        embedding=OpenAIEmbeddings(),
    )
    retriever = vectorstore.as_retriever()

    print(retriever.get_relevant_documents("What are Regular Expressions? Give me some ways I can use regular expressions?"))
    exit(0)

    prompt_template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | local_llm
        | StrOutputParser()
    )
    result = chain.invoke("What are Regular Expressions? Give me some ways I can use regular expressions?")
    print(result)

In [13]:
rag(documents, collection_name="test-x-3")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[
    Document(
        page_content='Regular expression is often shortened to regex. Regular expression is a language for 
specifying text search regular expression strings. Regular expression is used in every computer language. Regular 
expression is used in every word processor. Regular expression is used in text processing tools like the Unix tools
grep or Emacs. Formally, a regular expression is an algebraic notation for characterizing a set of strings. Regular
expressions are particularly useful for searching in texts. Regular expressions are used when there is a pattern to
search for. Regular expressions are used when there is a corpus of texts to search through. A regular expression 
search function will search through the corpus. A regular expression search function returns all texts that match 
the pattern. The Unix command-line tool grep takes a regular expression. The Unix command-line tool grep returns 
every line of the input document that matches the expression. A search can be designed to return every match on a 
line if there are more than one. A search can be designed to return just the first match on a line. In the 
following examples, the exact part of the pattern that matches the regular expression is generally underlined. In 
the following examples, only the first match is shown. Regular expressions in the examples are delimited by 
slashes. Slashes are not part of the regular expressions. Regular expressions come in many variants. We will be 
describing extended regular expressions. Different regular expression parsers may only recognize subsets of 
extended regular expressions. Different regular expression parsers may treat some expressions slightly differently.
Using an online regular expression tester is a handy way to test out your regular expressions. Using an online 
regular expression tester is a handy way to explore variations of regular expressions.',
        metadata={'source': 'local'}
    ),
    Document(
        page_content='One of the unsung successes in standardization in computer science has been the regular 
expression.',
        metadata={'source': 'local'}
    ),
    Document(
        page_content='The corpus can be a single document. The corpus can be a collection.',
        metadata={'source': 'local'}
    )
]

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Regular expressions are a language for specifying text search patterns. They are used in various computer 
languages, word processors, and text processing tools like Unix's grep or Emacs. Regular expressions are useful 
when there is a pattern to search for in a corpus of texts. A regular expression search function will look through 
the corpus and return all texts that match the specified pattern. For example, the Unix command-line tool "grep" 
takes a regular expression as an argument and returns every line from the input document that matches the 
expression. Regular expressions can be used to find multiple matches on a single line or just the first match. They
come in many variants, with extended regular expressions being one of them. Different parsers may recognize 
different subsets of extended regular expressions or treat some expressions slightly differently. An online regular
expression tester is a handy tool for testing and exploring variations of regular expressions.